In [55]:
import os
import pandas as pd
from Bio import PDB

def extract_chain_ids(pdb_file, molecular_name):
    """
    Extracts chain IDs from a PDB file that correspond to a given molecular name.
    
    Parameters:
    - pdb_file (str): Path to the PDB file.
    - molecular_name (str): Molecular name to search for in the PDB file.
    
    Returns:
    - List of chain IDs containing the specified molecular name.
    """
    chain_ids = set()  # Use a set to automatically handle duplicates
    with open(pdb_file, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            if molecular_name in line:
                # The line immediately after contains chain IDs
                if i + 1 < len(lines):
                    chain_line = lines[i + 1]
                    if "CHAIN:" in chain_line:
                        # Strip whitespace and semicolons, then split by commas
                        ids = chain_line.split("CHAIN:")[1].strip().strip(';').split(',')
                        chain_ids.update(id.strip(';').strip() for id in ids)
    chain_ids_list = list(chain_ids)
    print(f"Extracted chain IDs for {molecular_name} in {pdb_file}: {chain_ids_list}")
    return chain_ids_list

def get_dbref_displacement(pdb_file):
    """
    Extracts the displacement of residue IDs from the DBREF line in a PDB file.
    
    Parameters:
    - pdb_file (str): Path to the PDB file.
    
    Returns:
    - A dictionary mapping chain IDs to their residue ID displacements.
    """
    displacement = {}
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith("DBREF"):
                parts = line.split()
                chain_id = parts[2]
                pdb_residue_start = int(parts[3])
                seq_residue_start = int(parts[8])
                displacement[chain_id] = seq_residue_start - pdb_residue_start
    return displacement

def extract_residue_info(pdb_file, chain_ids):
    """
    Extracts residue information from a PDB file for specified chains, accounting for DBREF displacement.
    
    Parameters:
    - pdb_file (str): Path to the PDB file.
    - chain_ids (list): List of Chain IDs to filter residues.
    
    Returns:
    - List of dictionaries containing residue information.
    """
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('structure', pdb_file)
    residues = []

    displacement = get_dbref_displacement(pdb_file)

    for model in structure:
        for chain in model:
            if chain.get_id() in chain_ids:
                chain_displacement = displacement.get(chain.get_id(), 0)
                for residue in chain:
                    if PDB.is_aa(residue):
                        residue_id = residue.get_id()[1] + chain_displacement
                        residues.append({
                            'residue_id': residue_id,
                            'residue_name': residue.get_resname()
                        })
    
    # Debug output
    print(f"Extracted residues from {pdb_file}: {residues}")
    print(f"Type of residues: {type(residues)}")
    
    return residues

def compare_seq(ref_seq_file, pdb_file_paths, molecular_name, gene_name):
    """
    Compares and aligns sequences from reference and PDB files, and writes results to CSV.
    
    Parameters:
    - ref_seq_file (str): Path to the reference PDB file.
    - pdb_file_paths (list of str): List of PDB file paths to compare.
    - molecular_name (str): Molecular name to search for chain IDs.
    - gene_name (str): Gene name for output
    """
    # Create the output directory if it doesn't exist
    os.makedirs(f'{gene_name}_outputs', exist_ok=True)

    # Extract chain IDs for the reference sequence
    ref_chain_ids = extract_chain_ids(ref_seq_file, molecular_name)
    print(f"Reference Chain IDs: {ref_chain_ids}")

    # Extract residue information from the reference sequence
    ref_residues = extract_residue_info(ref_seq_file, chain_ids=ref_chain_ids)

    # Ensure ref_residues is a list of dictionaries
    if not isinstance(ref_residues, list):
        raise TypeError(f"Expected ref_residues to be a list of dictionaries, got {type(ref_residues)}")
    
    if len(ref_residues) == 0:
        raise ValueError("No residues extracted from reference sequence. Please check the PDB file and chain IDs.")

    # Extract residue information from each PDB file
    pdb_sequences = {}
    for pdb_file in pdb_file_paths:
        pdb_title = os.path.basename(pdb_file).replace('.pdb', '')  # Use the filename as the title
        chain_ids = extract_chain_ids(pdb_file, molecular_name)
        print(f"Chain IDs for {pdb_title}: {chain_ids}")

        # Extract residue info and categorize by chain ID
        chain_residues = {}
        for chain_id in chain_ids:
            chain_residues[chain_id] = extract_residue_info(pdb_file, chain_ids=[chain_id])
        
        pdb_sequences[pdb_title] = chain_residues

    # Align the sequences
    df = align_sequences(ref_residues, pdb_sequences)
    print("Aligned DataFrame:")
    print(df)

    # Output the DataFrame to a CSV file
    output_file = os.path.join(f'{gene_name}_outputs', f'{gene_name}_aligned_sequences_new.csv')
    df.to_csv(output_file, index=False)
    print(f"Aligned sequences have been written to {output_file}")

def align_sequences(ref_seq, pdb_sequences):
    """
    Aligns sequences from the reference and PDB sequences based on residue IDs,
    and sorts the columns by chain ID for each PDB file.
    
    Parameters:
    - ref_seq (list of dicts): Reference sequence residues.
    - pdb_sequences (dict): Dictionary with PDB sequence titles and chain residues.
    
    Returns:
    - Pandas DataFrame with aligned sequences, with columns sorted alphabetically by chain ID.
    """
    # Extract residue IDs from the reference sequence
    ref_residue_ids = [residue['residue_id'] for residue in ref_seq]

    # Initialize the DataFrame with the reference sequence
    df = pd.DataFrame(ref_residue_ids, columns=['residue_id'])
    df['ref_seq'] = [residue['residue_name'] for residue in ref_seq]

    # Collect columns to be added to DataFrame
    columns_to_add = []

    # Align each PDB sequence to the reference sequence
    for pdb_title, chain_residues in pdb_sequences.items():
        for chain_id, residues in chain_residues.items():
            pdb_dict = {res['residue_id']: res['residue_name'] for res in residues}
            column_name = f"{pdb_title}_{chain_id}"
            df[column_name] = df['residue_id'].apply(lambda x: pdb_dict.get(x, '-'))
            columns_to_add.append(column_name)

    # Sort columns alphabetically, keeping 'residue_id' and 'ref_seq' at the beginning
    sorted_columns = sorted(columns_to_add)
    sorted_columns = ['residue_id', 'ref_seq'] + sorted_columns
    df = df[sorted_columns]

    return df


In [56]:
ref_seq = './CHOA_pdb/AF-P12676-F1-model_v4.pdb'

pdb_file_paths = ['./CHOA_pdb/pdb1b4v.pdb', './CHOA_pdb/pdb1b8s.pdb', './CHOA_pdb/pdb1cbo.pdb', './CHOA_pdb/pdb1cc2.pdb', 
'./CHOA_pdb/pdb1ijh.pdb', './CHOA_pdb/pdb1mxt.pdb', './CHOA_pdb/pdb1n1p.pdb', './CHOA_pdb/pdb1n4u.pdb', 
'./CHOA_pdb/pdb1n4v.pdb', './CHOA_pdb/pdb1n4w.pdb', './CHOA_pdb/pdb2gew.pdb', './CHOA_pdb/pdb3b3r.pdb', 
'./CHOA_pdb/pdb3b6d.pdb', './CHOA_pdb/pdb3cnj.pdb', './CHOA_pdb/pdb3gyi.pdb', './CHOA_pdb/pdb3gyj.pdb', 
'./CHOA_pdb/pdb4rek.pdb', './CHOA_pdb/pdb4u2l.pdb', './CHOA_pdb/pdb4u2s.pdb', './CHOA_pdb/pdb4u2t.pdb', 
'./CHOA_pdb/pdb4xwr.pdb', './CHOA_pdb/pdb4xxg.pdb', './CHOA_pdb/pdb5kwf.pdb']

compare_seq(ref_seq, pdb_file_paths, "CHOLESTEROL OXIDASE","")

Extracted chain IDs for CHOLESTEROL OXIDASE in ./CHOA_pdb/AF-P12676-F1-model_v4.pdb: ['A']
Reference Chain IDs: ['A']
Extracted residues from ./CHOA_pdb/AF-P12676-F1-model_v4.pdb: [{'residue_id': 1, 'residue_name': 'MET'}, {'residue_id': 2, 'residue_name': 'THR'}, {'residue_id': 3, 'residue_name': 'ALA'}, {'residue_id': 4, 'residue_name': 'GLN'}, {'residue_id': 5, 'residue_name': 'GLN'}, {'residue_id': 6, 'residue_name': 'HIS'}, {'residue_id': 7, 'residue_name': 'LEU'}, {'residue_id': 8, 'residue_name': 'SER'}, {'residue_id': 9, 'residue_name': 'ARG'}, {'residue_id': 10, 'residue_name': 'ARG'}, {'residue_id': 11, 'residue_name': 'ARG'}, {'residue_id': 12, 'residue_name': 'MET'}, {'residue_id': 13, 'residue_name': 'LEU'}, {'residue_id': 14, 'residue_name': 'GLY'}, {'residue_id': 15, 'residue_name': 'MET'}, {'residue_id': 16, 'residue_name': 'ALA'}, {'residue_id': 17, 'residue_name': 'ALA'}, {'residue_id': 18, 'residue_name': 'PHE'}, {'residue_id': 19, 'residue_name': 'GLY'}, {'residue_i